# Chapter 5 - Resampling Methods: Applied Exercises

In [1]:
# import packages
import pandas as pd

### Exercise 5

In [2]:
default = pd.read_csv('data/Default.csv')
default.head() # remove unnamed column

,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.625074
1,2,No,Yes,817.180407,12106.134700
2,3,No,No,1073.549164,31767.138947
3,4,No,No,529.250605,35704.493935
4,5,No,No,785.655883,38463.495879


In [ ]:
# a) fit logReg -> default ~ income + balance

In [ ]:
# b) estimate test error
# i) train / test split
# ii) Fit multiple logReg
# iii) obtain a prediction of default status for each individual in the validation set
# iv) compute the validation set error

In [ ]:
# c) repeat b) three times using three different splits

In [ ]:
# d) fit logReg -> default ~ income + balance + student(0/1) and estimate test error

### Exercise 6

In [ ]:
# a) determine estimated SE for coefficients (logReg -> default ~ income + balance)

In [ ]:
# b) write boot.fn() function that outputs the coefficient estimates for income and balance

In [ ]:
# c) use bootstrap to estimate the SEs of the logReg coefficients 

In [ ]:
# d) comment on the estimated standard errors obtained

### Exercise 7

In [3]:
weekly = pd.read_csv('data/Weekly.csv')
weekly.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [ ]:
# a) fit logReg -> Direction ~ Lag1 + Lag2

In [ ]:
# b) fit logReg -> Direction ~ Lag1 + Lag2 with all but the first observation 

In [ ]:
# c) use the model from b= to predict the direction of the first observation; was it classified correctly?

In [ ]:
# d) for loop i-n:
# i) fit a logReg 
# ii) compute the posterior probability of the market moving up for the ith observation
# iii) use the posterior probability for the ith observation in order to predict whether or not the market moves up
# iv) Determine whether or not an error was made in predicting the direction for the ith observation (1 = error, 0 = none)

In [ ]:
# e) take the avg of the n numbers obtained in d) in order to obtain the LOOCV estimate for the test error

### Exercise 8

In [ ]:
# a) generate simulated data

In [ ]:
# b) scatterplot of X against Y

In [ ]:
# c) set a random seed and compute the LOOCV errors
# i), ii), iii), iv)

In [ ]:
# d) repeat c) using a different seed

In [ ]:
# e) which of the models in c) had the smallest LOOCV error?

In [ ]:
# f) comment on the statistical significance of the coefficients in c)

### Exercise 9

In [4]:
boston = pd.read_csv('data/Boston.csv')
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [ ]:
# a) estimate population mean of medv

In [ ]:
# b) provide an estimate of the standard error of a)

In [ ]:
# c) estimate SE of a) using bootstrap

In [ ]:
# d) provide a 95% CI based on c) and compare it to t-Test

In [ ]:
# e) provide an estimate for the median of medv

In [ ]:
# f) estimate SE of median e) using bootstrap

In [ ]:
# g) provide an estimate for tenth percentile of medv

In [ ]:
# h) use bootstrap to estimate SE of g)